In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

자료를 벡터 db에 넣기

In [2]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 프롬프트

In [17]:
SYS_PROMPT = f"""
    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자의 수강정보는 강남대학교의 졸업요건과 함께 데이터베이스에 같이 저장되어 있어.
    교양 ,주전공, 복수전공, 등을 나눠서 설명해줘야해
    주전공에서 전기는 전공 기초고 전선은 전공 선택이야. 
    복수전공에서 복기는 복수전공 기초고 복선은 복수전공 선택이야.
    마이크로 전공은 부가정보야.
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요
"""

모델 선언

In [18]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''Answer the question based only on the following context:
{context}

Question: {question}
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
내가 졸업할수있는지 알려줘.
"""   # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke("input_message"))


> 우승우 학생의 졸업 가능 여부를 확인하기 위해서는 다음과 같은 사항들을 검토해야 합니다:
> 
> ### 교양 과목
> - **졸업 기준**: 17학점
> - **취득 학점**: 17학점
> - **미취득 학점**: 0학점
> 
> ### 전공 과목
> - **인공지능전공 (제1전공)**: 
>   - **취득 학점**: 12학점
>   - **미취득 학점**: 3학점
> - **소프트웨어전공 (복수전공)**:
>   - **취득 학점**: 18학점
>   - **평균 평점**: 4.37
> 
> ### 마이크로 전공
> - **기계학습**:
>   - **취득 학점**: 9학점
>   - **미이수 학점**: 3학점
> 
> ### 사회봉사 인증
> - **상태**: 대상 (인증 필요)
> 
> ### 전체 학점 및 평점
> - **전체 취득 학점**: 74학점
> - **전체 평점 평균**: 4.30
> 
> ### 결론
> 우승우 학생은 현재까지 74학점을 취득하였으며, 전체 평점 평균은 4.30입니다. 그러나 다음 사항들이 충족되어야 졸업이 가능합니다:
> 1. **사회봉사 인증**: 현재 '대상' 상태로, '인증' 상태로 변경되어야 합니다.
> 2. **마이크로 전공**: 기계학습 마이크로 전공에서 3학점을 추가로 이수해야 합니다.
> 3. **전공 필수 학점**: 인공지능전공에서 미취득한 3학점을 이수해야 합니다.
> 
> 이 모든 요건을 충족하면 졸업이 가능합니다. 추가로 궁금한 사항이 있으면 언제든지 문의해 주세요.